In [2]:
#step 1: Set Goals, if you haven't yet, go back!
#      For this sample, the goal was create a sales forecast for the entire store branches including the main store.
"""
US Candy Distributor
Sales and geospatial factory to customer shipment data for a US national candy distributor, inlcuding information around customer & factory locations, sales orders & goals, and product details.

Recommended Cmpt
What are the most efficient factory to customer shipping routes?
- shipping routes, factory location(lat.&lon.) & customer location(lat.&long) calculate distance with c^2 = a^2 + b^2
- we need Sales/Gross Profit to calculate priorities for actions.

What about the least efficient?
- shipping routes, factory location(lat.&lon.) & customer location(lat.&long) calculate distance with c^2 = a^2 + b^2

Which product lines have the best product margin?

Which product lines should be moved to a different factory to optimize shipping routes?

Want feedback on your solutions?
Share visualizations (and any applicable pivot tables, code, etc.) on LinkedIn and mention @Maven Analytics. We would love to see your work and give our thoughts!
"""

#Data integration 
#       Candy_Sales.Division -> Candy_Targets.Division
#       Candy_Sales.Postal Codes -> uszips.zip
#       Candy_Sales.Product ID -> Candy_Products.Product ID
#       Candy_Products.Factory -> Cacndy_Factories.Factory

#Step 2: Data Preparation
import pandas as pd


In [3]:
#loading data to df_orig - take note that the following code is only for .csv file data that I already 
#   taken a glimpse on the content thus, I was able to specify header, parse_dates, index_col and dayfirst
#   else you need to take that move first or you can reload the data file after analyzing it here if you want.
#step 2.a: Grab Data

#df_orig = pd.read_csv('Candy_Sales.csv', header = 0, parse_dates=['OrderDate'], index_col='date', dayfirst=True)
"""
df_orig_Candy_Sales = pd.read_csv('Candy_Sales.csv', header = 0)
df_orig_uszips= pd.read_csv('uszips.csv', header = 0)
"""
candy_sales = pd.read_csv('Candy_Sales.csv')
candy_targets = pd.read_csv('Candy_Targets.csv')
uszips = pd.read_csv('uszips.csv')
candy_products = pd.read_csv('Candy_Products.csv')
candy_factories = pd.read_csv('Candy_Factories.csv')

In [4]:
candy_sales['Postal Code'] = pd.to_numeric(candy_sales['Postal Code'], errors='coerce', downcast='integer')

In [5]:
# Merge tables
merged_df = (candy_sales
    .merge(candy_targets, on='Division')
    .merge(uszips, left_on='Postal Code', right_on='zip')
    .merge(candy_products, on='Product ID')
    .merge(candy_factories, left_on='Factory', right_on='Factory')
)

In [6]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 44 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Row ID            9994 non-null   int64  
 1   Order ID          9994 non-null   object 
 2   Order Date        9994 non-null   object 
 3   Ship Date         9994 non-null   object 
 4   Ship Mode         9994 non-null   object 
 5   Customer ID       9994 non-null   int64  
 6   Country/Region    9994 non-null   object 
 7   City              9994 non-null   object 
 8   State/Province    9994 non-null   object 
 9   Postal Code       9994 non-null   float64
 10  Division_x        9994 non-null   object 
 11  Region            9994 non-null   object 
 12  Product ID        9994 non-null   object 
 13  Product Name_x    9994 non-null   object 
 14  Sales             9994 non-null   float64
 15  Units             9994 non-null   int64  
 16  Gross Profit      9994 non-null   float64


In [7]:
final_df = merged_df[['Order ID','Division_y', 'Postal Code', 'Product ID', 'Factory','lat','lng','Latitude','Longitude','Sales','Cost','Units','Gross Profit','Unit Cost']].copy()

In [8]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Order ID      9994 non-null   object 
 1   Division_y    9994 non-null   object 
 2   Postal Code   9994 non-null   float64
 3   Product ID    9994 non-null   object 
 4   Factory       9994 non-null   object 
 5   lat           9994 non-null   float64
 6   lng           9994 non-null   float64
 7   Latitude      9994 non-null   float64
 8   Longitude     9994 non-null   float64
 9   Sales         9994 non-null   float64
 10  Cost          9994 non-null   float64
 11  Units         9994 non-null   int64  
 12  Gross Profit  9994 non-null   float64
 13  Unit Cost     9994 non-null   float64
dtypes: float64(9), int64(1), object(4)
memory usage: 1.1+ MB


In [9]:
duplicate_count = final_df['Order ID'].duplicated().sum()

In [10]:
duplicate_rows = final_df[final_df['Order ID'].duplicated(keep=False)]
print(duplicate_rows)

                          Order ID Division_y  Postal Code     Product ID  \
9     US-2024-147039-CHO-NUT-13000  Chocolate      55407.0  CHO-NUT-13000   
10    US-2024-147039-CHO-NUT-13000  Chocolate      55407.0  CHO-NUT-13000   
18    US-2023-131065-CHO-NUT-13000  Chocolate      30318.0  CHO-NUT-13000   
20    US-2023-131065-CHO-NUT-13000  Chocolate      30318.0  CHO-NUT-13000   
23    US-2024-100230-CHO-NUT-13000  Chocolate      10035.0  CHO-NUT-13000   
...                            ...        ...          ...            ...   
9984  US-2024-100013-CHO-SCR-58000  Chocolate      90045.0  CHO-SCR-58000   
9985  US-2021-143336-CHO-TRI-54000  Chocolate      94109.0  CHO-TRI-54000   
9986  US-2021-143336-CHO-TRI-54000  Chocolate      94109.0  CHO-TRI-54000   
9991  US-2023-152471-CHO-TRI-54000  Chocolate      32216.0  CHO-TRI-54000   
9992  US-2023-152471-CHO-TRI-54000  Chocolate      32216.0  CHO-TRI-54000   

              Factory       lat        lng   Latitude   Longitude  Sales  \

In [11]:
final_df.rename(columns={
    'Division_y': 'Division',
    'Postal Code': 'Postal_Code',
    'Product ID': 'Product_ID',
    'Factory': 'Factory',
    'lat': 'Cust_Lat',
    'lng': 'Cust_Long',
    'Latitude': 'Fact_Lat',
    'Longitude': 'Fact_Long',
    'Cost': 'Cost',
    'Gross Profit': 'Gross_Profit',
    'Units': 'Units',
    'Unit Cost': 'Unit_Cost'
}, inplace=True)

In [12]:
# (STARTS HERE)Calculation for Product Margin

factory = final_df.groupby('Product_ID').agg({
    'Product_ID' : 'first' ,
    'Factory': 'first',
    'Sales': 'sum',
    'Units': 'sum',
    'Gross_Profit': 'sum'
})

Cmpt_Product_Margin = factory.reset_index(drop=True)
Cmpt_Product_Margin['Sales_pU'] = Cmpt_Product_Margin['Sales'] / Cmpt_Product_Margin['Units']
Cmpt_Product_Margin['Gross_pU'] = Cmpt_Product_Margin['Gross_Profit'] / Cmpt_Product_Margin['Units']
Cmpt_Product_Margin['Product_Margin'] = (Cmpt_Product_Margin['Gross_pU'] / Cmpt_Product_Margin['Sales_pU'])*100

# (ENDS HERE)Calculation for Product Margin

In [13]:
import pandas as pd
from geopy.distance import great_circle

# Calculate distances and cost per unit
final_df['cost_per_units'] = final_df['Cost'] / final_df['Units']


def calculate_distance1(row):
    cust_coords = (row['Cust_Lat'], row['Cust_Long'])
    fact_coords = (row['Fact_Lat'], row['Fact_Long'])
    return great_circle(cust_coords, fact_coords).kilometers
    
final_df['Distance'] = final_df.apply(calculate_distance1, axis=1)





In [14]:
# (STARTS HERE)Calculation for ORIGNAL Units Per Distance 
final_df['units_per_distance_orig'] = final_df['Distance'] / final_df['Units']
Cmpt_Distance_mean_byproduct = final_df.groupby(final_df['Product_ID'])['units_per_distance_orig'].mean()
Cmpt_Distance_mean_byproduct = pd.DataFrame(Cmpt_Distance_mean_byproduct).reset_index()
# (ENDS HERE)Calculation for ORIGNAL Units Per Distance 

In [15]:
# (STARTS HERE)Calculate Distance Score per order
# Define factory coordinates
factories = {
    'lot': (32.881893, -111.768036),
    'wic': (32.076176, -81.088371),
    'sug': (48.11914, -96.18115),
    'sec': (41.446333, -90.565487),
    'the': (35.1175, -89.971107)
}


def calculate_distance(row, factory_coords):
    customer_coords = (row['Cust_Lat'], row['Cust_Long'])
    return great_circle(customer_coords, factory_coords).kilometers

# Calculate distances to all factories
for key, coords in factories.items():
    final_df[f'to_{key}'] = final_df.apply(calculate_distance, axis=1, factory_coords=coords)
    #final_df[f'dist_to_{key}'] = final_df.apply(calculate_distance, axis=1, factory_coords=coords)


# Function to assign scores based on distance and sales volume
def assign_scores(row, distance_columns, sales_volume):
    distances = row[distance_columns]
    sorted_indices = distances.rank(method='min').astype(int)
    base_scores = 6 - sorted_indices  # Assign 5 for the nearest, 1 for the furthest
    # Adjust scores by sales volume
    adjusted_scores = row[distance_columns] / sales_volume
    return adjusted_scores


# 
distance_columns = [f'to_{key}' for key in factories.keys()]
final_df['Sales_Volume'] = final_df['Units']  # Assuming 'Units' represents sales volume

for key in factories.keys():
    final_df[f'score_{key}'] = final_df.apply(
        lambda row: assign_scores(row, distance_columns, row['Sales_Volume'])[f'to_{key}'], axis=1
    )
# Sum scores by factory
score_columns = [f'score_{key}' for key in factories.keys()]
factory_scores = final_df[score_columns].sum()


Cmpt_Distance_Optimal_Factory_ByProduct = final_df.groupby(final_df['Product_ID']).agg({
    'Factory' : 'first',
    'score_lot': 'mean',
    'score_wic': 'mean',
    'score_sug': 'mean',
    'score_sec': 'mean',
    'score_the': 'mean'
})
# (ENDS HERE)Calculate Distance Score per order


In [16]:
# Create the DataFrame and set the index for suggested location.
df_suggest_loc= pd.DataFrame(Cmpt_Distance_Optimal_Factory_ByProduct)

# Set Product_ID as the index
df_suggest_loc.reset_index(inplace=True)
df_suggest_loc.set_index('Product_ID', inplace=True)

# Define the factories dictionary
factories = {
    'score_lot': (32.881893, -111.768036),
    'score_wic': (32.076176, -81.088371),
    'score_sug': (48.11914, -96.18115),
    'score_sec': (41.446333, -90.565487),
    'score_the': (35.1175, -89.971107)
}

# Define the factory recommendations
factory_recommend = {
    'score_lot': "Lot's O' Nuts",
    'score_wic': "Wicked Choccy's",
    'score_sug': "Sugar Shack",
    'score_sec': "Secret Factory",
    'score_the': "The Other Factory"
}

# Convert the factories dictionary to a DataFrame
factories_df = pd.DataFrame(factories, index=['latitude', 'longitude']).T

# Add latitude and longitude columns to the original DataFrame
for score_type in factories.keys():
    df_suggest_loc[f'{score_type}_latitude'] = factories[score_type][0]
    df_suggest_loc[f'{score_type}_longitude'] = factories[score_type][1]

# Automatically find the column with the highest value for each Product_ID
score_columns = ['score_lot', 'score_wic', 'score_sug', 'score_sec', 'score_the']
df_suggest_loc['highest_score_column'] = df_suggest_loc[score_columns].idxmin(axis=1)

# Add coordinates for the column with the highest score as a string
df_suggest_loc['highest_score_column'] = df_suggest_loc[['score_lot', 'score_wic', 'score_sug', 'score_sec', 'score_the']].idxmin(axis=1)
df_suggest_loc['highest_score_coordinates'] = df_suggest_loc['highest_score_column'].apply(lambda col: f"{factories[col][0]},{factories[col][1]}")

# Add recommended factory for the highest score column
df_suggest_loc['recommended_factory'] = df_suggest_loc['highest_score_column'].map(factory_recommend)


import pandasql as psql

query="""
SELECT Product_ID,
    Factory AS Factory_Origin,
    recommended_factory AS Factory_Suggested,
    highest_score_coordinates AS Coord_Suggested
FROM df_suggest_loc
ORDER BY Product_ID
"""

Suggested_Location = psql.sqldf(query, locals())
Suggested_Location.set_index('Product_ID', inplace=True)


In [17]:

# Function to extract coordinates from the coordinate string
def extract_coordinates(coord_str):
    lat, long = map(float, coord_str.split(','))
    return (lat, long)

# Function to get suggested factory coordinates
def get_suggested_fact_coord(product_id):
    if product_id in Suggested_Location.index:
        row = Suggested_Location.loc[product_id]
        coord_str = row['Coord_Suggested']
        return extract_coordinates(coord_str)
    return (None, None)

# Function to calculate the distance
def calculate_distance_move(row): 
    cust_coords = (row['Cust_Lat'], row['Cust_Long'])
    fact_coords = get_suggested_fact_coord(row['Product_ID'])
    
    if fact_coords[0] is None or fact_coords[1] is None:
        return None  # Or a default value if the coordinates are not found
    
    return great_circle(cust_coords, fact_coords).kilometers

# Apply the function to each row
final_df['Suggest_Dist'] = final_df.apply(calculate_distance_move, axis=1)

final_df['Suggest_Fact'] = final_df['Product_ID'].map(Suggested_Location['Factory_Suggested'])

Cmpt_Suggest_Dist = final_df.groupby(final_df['Product_ID'])['Suggest_Dist'].mean()

Cmpt_Total_Sales = final_df.groupby(final_df['Product_ID']).agg({
    'Sales': 'sum',
    'Distance': 'sum',
    'Suggest_Dist': 'sum',
    'Factory' : 'first'
})

In [18]:
#Preparing for graph
import pandasql as psql

query = """
SELECT
    Product_ID,
    Factory,
    AVG(to_lot) AS "Lot's O' Nuts",
    AVG(to_wic) AS "Wicked Choccy's",
    AVG(to_sug) AS "Sugar Shack",
    AVG(to_sec) AS "Secret Factory",
    AVG(to_the) AS "The Other Factory"
FROM final_df
GROUP BY Product_ID
"""

df_pie_ = psql.sqldf(query, locals())
df_pie_.set_index('Product_ID', inplace=True)

query = """
SELECT
    Product_ID AS product_id,
    AVG(Suggest_Dist) AS distance_new,
    AVG(Distance) AS distance_orig
FROM final_df
GROUP BY Product_ID
ORDER BY product_id
"""

df_to_melt = psql.sqldf(query, locals())

# Melt the DataFrame to long format
df_melted = df_to_melt.melt(id_vars='product_id', value_vars=['distance_orig', 'distance_new'],
                             var_name='distance_type', value_name='distance_value')


#Normalizing df_melted
# Apply min-max normalization
min_value = df_melted['distance_value'].min()
max_value = df_melted['distance_value'].max()

df_melted['normalized_value'] = (df_melted['distance_value'] - min_value) / (max_value - min_value)


In [27]:
# Just a test. 
query1 = """
SELECT
    Factory,
    Factory_Suggested,
    Units,
    score_lot,
    score_wic,
    score_sug,
    score_sec,
    score_the,
    Distance as Old_Dist,
    Suggest_Dist AS New_Dist,
    Postal_Code
FROM final_df
INNER JOIN Suggested_Location on final_df.Product_ID = Suggested_Location.Product_ID
WHERE final_df.Product_ID = "SUG-FUN-75000"
"""
query2 = """
SELECT
    Factory,
    Factory_Suggested,
    Units,
    to_lot,
    to_wic,
    to_sug,
    to_sec,
    to_the,
    Distance as Old_Dist,
    Suggest_Dist AS New_Dist,
    Postal_Code
FROM final_df
INNER JOIN Suggested_Location on final_df.Product_ID = Suggested_Location.Product_ID
WHERE final_df.Product_ID = "SUG-FUN-75000"
"""
test_it = psql.sqldf(query1, locals())

print(test_it.head(50))

       Factory  Factory_Suggested  Units       to_lot       to_wic  \
0  Sugar Shack  The Other Factory      4  3445.327925  1152.649783   
1  Sugar Shack  The Other Factory      1  2459.083350   499.598762   
2  Sugar Shack  The Other Factory      3  2337.014672  1234.963294   

        to_sug       to_sec       to_the     Old_Dist     New_Dist  \
0  1933.710115  1388.245527  1530.988259  1933.710115  1530.988259   
1  1716.593188   855.508360   438.534200  1716.593188   438.534200   
2   962.909812   240.803180   773.586826   962.909812   773.586826   

   Postal_Code  
0      10011.0  
1      37421.0  
2      60623.0  


In [28]:
query = """
SELECT
    Cmpt_Distance_Optimal_Factory_ByProduct.Product_ID,
    Factory,
    (score_lot + score_wic + score_sug + score_sec + score_the) - score_lot AS "Lot's O' Nuts",
    (score_lot + score_wic + score_sug + score_sec + score_the) - score_wic AS "Wicked Choccy's",
    (score_lot + score_wic + score_sug + score_sec + score_the) - score_sug AS "Sugar Shack",
    (score_lot + score_wic + score_sug + score_sec + score_the) - score_sec AS "Secret Factory",
    (score_lot + score_wic + score_sug + score_sec + score_the) - score_the AS "The Other Factory"
FROM Cmpt_Distance_Optimal_Factory_ByProduct
"""

try_this = psql.sqldf(query, locals())
try_this.set_index('Product_ID', inplace=True)

query = """
SELECT
    Cmpt_Distance_Optimal_Factory_ByProduct.Product_ID,
    Factory,
    Factory_Suggested
FROM Cmpt_Distance_Optimal_Factory_ByProduct
INNER JOIN Suggested_Location on Cmpt_Distance_Optimal_Factory_ByProduct.Product_ID = Suggested_Location.Product_ID
"""

for_app_action = psql.sqldf(query, locals())
for_app_action.set_index('Product_ID', inplace=True)


In [29]:
print(for_app_action.head(20))
print(try_this)
print(df_pie_)

                         Factory  Factory_Suggested
Product_ID                                         
CHO-FUD-51000      Lot's O' Nuts     Secret Factory
CHO-MIL-31000    Wicked Choccy's     Secret Factory
CHO-NUT-13000      Lot's O' Nuts     Secret Factory
CHO-SCR-58000      Lot's O' Nuts     Secret Factory
CHO-TRI-54000    Wicked Choccy's     Secret Factory
OTH-FIZ-56000        Sugar Shack     Secret Factory
OTH-GUM-21000     Secret Factory     Secret Factory
OTH-KAZ-38000  The Other Factory     Secret Factory
OTH-LIC-15000     Secret Factory     Secret Factory
SUG-EVE-47000     Secret Factory  The Other Factory
SUG-FUN-75000        Sugar Shack  The Other Factory
SUG-HAI-55000  The Other Factory  The Other Factory
SUG-LAF-25000        Sugar Shack     Secret Factory
SUG-NER-92000        Sugar Shack    Wicked Choccy's
SUG-SWE-91000        Sugar Shack  The Other Factory
                         Factory  Lot's O' Nuts  Wicked Choccy's  Sugar Shack  \
Product_ID                         

In [32]:
#BELOW STARTS DASHBOARD CREATION#

import plotly.express as px
import dash
from dash import html, dcc
import pandas as pd  # Ensure pandas is imported

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

factory_colors = {
    "Lot's O' Nuts": 'blue',
    "Wicked Choccy's": 'red',
    "Sugar Shack": 'green',
    "Secret Factory": 'black',
    "The Other Factory": 'gold'
}

def get_light_color(color):
    color_dict = {
        'blue': '#72bcd4',    # Light blue
        'red': '#ff817f',     # Light red
        'green': '#64e764',   # Light green
        'black': '#bababa',   # Light grey
        'gold': '#ffff94'     # Light yellow
    }
    return color_dict.get(color, 'gray')  # Default to gray if color not found

# Melt the DataFrame
data_melted = pd.melt(final_df, id_vars=['Division', 'Postal_Code', 'Product_ID', 'Factory'],
                      value_vars=['Cust_Lat', 'Fact_Lat'],
                      var_name='Location_Type', value_name='Latitude')
data_melted['Longitude'] = pd.melt(final_df, id_vars=['Division', 'Postal_Code', 'Product_ID', 'Factory'],
                                   value_vars=['Cust_Long', 'Fact_Long'],
                                   var_name='Location_Type', value_name='Longitude')['Longitude']

# Map location types and colors
data_melted['Location_Type'] = data_melted['Location_Type'].apply(lambda x: 'Customer' if 'Cust' in x else 'Factory')

# Create columns for legend display and colors
data_melted['Legend'] = data_melted.apply(
    lambda row: f'Sold From {row["Factory"]}' if row['Location_Type'] == 'Customer' else row['Factory'],
    axis=1
)

data_melted['Color'] = data_melted.apply(
    lambda row: get_light_color(factory_colors.get(row['Factory'], 'gray'))
    if row['Location_Type'] == 'Customer' else factory_colors.get(row['Factory'], 'gray'),
    axis=1
)

# Set text for factory locations only
data_melted['Text'] = data_melted.apply(lambda row: row['Factory'] if row['Location_Type'] == 'Factory' else '', axis=1)


# Create the Plotly figure
fig = px.scatter_mapbox(
    data_melted,
    lat='Latitude',
    lon='Longitude',
    color='Legend',
    color_discrete_map={legend: color for legend, color in zip(data_melted['Legend'].unique(), data_melted['Color'].unique())},
    text='Text',  # Set text only for factories
    mapbox_style='open-street-map',  # Simpler map style
    zoom=3,
    center={'lat': 37.7749, 'lon': -115.7129}
)

# Ensure the Plotly figure is also responsive
fig.update_layout(
    #autosize=True,
    margin=dict(l=0, r=0, t=0, b=0),  # Adjust margins
    legend=dict(
        title='Location Type',
        orientation='h',
        yanchor='bottom',
        y=0.02,
        xanchor='right',
        x=1
    )
)

fig.update_traces(marker=dict(size=12))  # Smaller markers

# Initialize the app
app = dash.Dash(__name__)

# Sample figures (replace with your actual figures)
fig1 = px.bar(
    Cmpt_Product_Margin,
    x='Product_ID',
    y='Product_Margin',
    title='Product Profit Margins'
)

fig2 = px.bar(
    df_melted,
    x='product_id',
    y='distance_value',
    color='distance_type',
    barmode='group',
    title='Comparison Distance of Original and Proposed location by Product ID'
)

# Define layout and callback
app.layout = html.Div([
    # Container for the heading
    html.Div(
        html.H1("Supply Chain Dashboard"),
        style={
            'display': 'flex',
            'justify-content': 'flex-end',
            'margin-bottom': '20px'  # Add some space below the heading
        }
    ),
    dcc.Dropdown(
        id='product-dropdown',
        options=[{'label': i, 'value': i} for i in try_this.index],
        value=try_this.index[0],  # Default value
        className='dropdown',
        style={'justify-content': 'space-between', 'width': '100%', 'margin-bottom': '30px'}
    ), 
    html.Div([
            dcc.Graph(
                id='route-map',
                figure=fig,
                style={'flex': '1', 'width': '100%', 'height': '550'}
            ),
            dcc.Graph(
                id='pie-chart',
                className='pie-chart',
                style={
                    'flex': '1',
                    'top': '-15%',
                    'left': '0%',
                    'width': '40%'} 
                    #'position': 'relative'  # Set relative position for overlaying elements
            ),
            dcc.Textarea(
                id='textarea',
                value='Textarea content initialized\nwith multiple lines of text',
                style={
                    'position': 'absolute',  # Overlay the textarea on the pie-chart
                    'top': '57%',  # Positioning the textarea
                    'left': '1%',
                    #'transform': 'translate(-50%, -50%)',  # Centering the textarea
                    'width': '40%',  # Set the width
                    'height': '20%',  # Set the height
                    'background-color': 'rgba(255, 255, 255, 0.7)',  # Semi-transparent background
                    #'z-index': '10',  # Ensure it stays on top
                }
            ),
             html.Div(
                    id='text-output'  # Add this component for displaying text output
            ),
       ], style={'position': 'relative'}),  # Container with relative position for overlay        
    html.Div([

    ], style={'display': 'flex', 'justify-content': 'space-between', 'width': '100%'}),
    
    html.Div([
        dcc.Graph(
            id='product-margin',
            figure=fig1,
            style={'flex': '1', 'margin-left': '10px'}
        ),
        dcc.Graph(
            id='relocation-analysis',
            figure=fig2,
            style={'flex': '1', 'margin-left': '10px'}
        )
    ], style={'display': 'flex', 'justify-content': 'space-between', 'width': '100%'}),
    
])

@app.callback(
    [dash.dependencies.Output('pie-chart', 'figure'),
     dash.dependencies.Output('text-output', 'children'),
     dash.dependencies.Output('textarea', 'value')],
    [dash.dependencies.Input('product-dropdown', 'value')]
)
def update_pie_chart(selected_product):
    # Extract information from for_app_action
    for_calc_ = df_pie_.loc[selected_product]
    # For Action additional info 
    for_action_ = for_app_action.loc[selected_product]
    for_action_df = pd.DataFrame(for_action_).reset_index()
    for_action_df.columns = ['Location_Type', 'Data']
        # For Action's recalculation
            
    
    # For plotting pie-chart
    filtered_series = try_this.loc[selected_product]
    filtered_df = pd.DataFrame(filtered_series).reset_index()
    filtered_df.columns = ['metric', 'value']
    
    # Create text output
    # Convert 'value' column to numeric, coercing errors (non-numeric) to NaN
    filtered_df['value'] = pd.to_numeric(filtered_df['value'], errors='coerce')
    
    # Drop rows where 'value' is NaN (if any)
    filtered_df = filtered_df.dropna(subset=['value'])
    
    # Create the pie chart
    fig3 = px.pie(filtered_df, names='metric', values='value', title=f'Scores for {selected_product}')
    fig3.update_layout(
        paper_bgcolor='rgba(0,0,0,0)',  # Transparent background
        plot_bgcolor='rgba(0,0,0,0)'    # Transparent background for the plot area
    )
    
    # Determine the recommended factory based on the pie chart result
    if filtered_df.empty:
        text_output = f'No valid data available for {selected_product}.'
    else:
        # Get the metric with the highest value
        total_value = filtered_df['value'].sum()
        max_value = filtered_df['value'].max()
        recommended_metric = filtered_df.loc[filtered_df['value'].idxmax(), 'metric']
        recommended_metric_percentage = (max_value / total_value) * 100

        factory_recommendation = {
            'Lot\'s O\' Nuts': "Lot's O' Nuts",
            'Wicked Choccy\'s': "Wicked Choccy's",
            'Sugar Shack': "Sugar Shack",
            'Secret Factory': "Secret Factory",
            'The Other Factory': "The Other Factory"
        }.get(recommended_metric, "Unknown Factory")
        
        factory_recommendation = factory_recommend.get(recommended_metric, "Unknown Factory")

        if for_action_df.loc[0,'Data']!=for_action_df.loc[1,'Data'] :
            
            action_1 = for_action_df.loc[for_action_df['Location_Type'] == 'Factory', 'Data'].iloc[0]
            action_2 = for_action_df.loc[for_action_df['Location_Type'] == 'Factory_Suggested', 'Data'].iloc[0]
            
            #for_calc_ = df_pie_.loc[selected_product]
            for_calc_data1 = for_calc_.loc[action_1]
            for_calc_data2 = for_calc_.loc[action_2]
            calc_ = ((for_calc_data1 - for_calc_data2)/for_calc_data1)*100

            text_output = (
                f'Selected Product: {selected_product} from {action_1} has current mean distance of {for_calc_data1: .2f} KM with the proposed location, it will be'
                f' around {for_calc_data2: .2f} KM.'
            ),
            textarea_ = (
                f'Selected Product: {selected_product} from {action_1} is recommended to {action_2} '
                f'with {recommended_metric_percentage:.1f}% of customer\'s favorable location and a possible {calc_:.1f}% improvement in shipping efficiency.'
            )
        else:
            action_1 = for_action_df.loc[for_action_df['Location_Type'] == 'Factory', 'Data'].iloc[0]
            text_output = (
                f'Selected Product: {selected_product} from {action_1} is already at its optimal location.'
            ), 
            textarea_ = (f'Selected Product: {selected_product} from {action_1} is already at its optimal location.'
            ) 
        
  
    return fig3, text_output , textarea_  # Return a tuple with the figure and text

if __name__ == '__main__':
    app.run_server(debug=True, port=7170)  # Use a different port number
